# TP1 – Analyse des données (IFT599 / IFT799)

**Auteurs :**  
-  
-  
-  3

**Date de remise :** 03 octobre 2025  


In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

### Charger tous les jeux de données

In [3]:
# Load transaction data
transactions = pd.read_csv("data/transactions_data.csv")

# Load card information
cards = pd.read_csv("data/cards_data.csv")

# Load user information
users = pd.read_csv("data/users_data.csv")

# Load mcc codes information
with open("data/mcc_codes.json", "r") as f:
    mcc_data = json.load(f)

mcc_codes = pd.DataFrame(list(mcc_data.items()), columns=["code", "description"])

# Load fraud information
with open("data/train_fraud_labels.json", "r") as f:
    fraud_data = json.load(f)

# Extract the inner dictionary
fraud_dict = fraud_data['target']

# Convert to DataFrame
fraud_labels = pd.DataFrame(list(fraud_dict.items()), columns=["transaction_id", "is_fraud"])

# 3.1 Préparation des données (25%)

**1. Fusion des différents fichiers** (`transactions`, `cards`, `users`, `mcc`, `fraud`)  
  - Clés de jointure utilisées  
  - Vérifier lees colonnes dupliquées et suffixes `_x`, `_y`  (causés par pandas)
  - Expliquer les choix 

In [4]:
# Rename keys before merging
transactions = transactions.rename(columns={"id": "transaction_id"})
cards = cards.rename(columns={"id": "card_id"})
users = users.rename(columns={"id": "user_id"})
mcc_codes = mcc_codes.rename(columns={"code": "mcc_code", "description": "mcc_description"})

In [5]:
# Ensure same dtype for transaction_id
transactions["transaction_id"] = transactions["transaction_id"].astype(str)
fraud_labels["transaction_id"] = fraud_labels["transaction_id"].astype(str)

# Ensure same dtype for mcc
transactions["mcc"] = transactions["mcc"].astype(str)
mcc_codes["mcc_code"] = mcc_codes["mcc_code"].astype(str)

In [6]:
merged_df = (
    transactions
    .merge(cards, on="card_id", how="left", suffixes=("", "_card"))
    .merge(users, left_on="client_id", right_on="user_id", how="left", suffixes=("", "_user"))
    .merge(mcc_codes, left_on="mcc", right_on="mcc_code", how="left")
    .merge(fraud_labels, on="transaction_id", how="left")
)

merged_df.head()

,transaction_id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,...,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,mcc_code,mcc_description,is_fraud
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,...,46.80,-100.76,$23679,$48277,$110153,740,4,5499,Miscellaneous Food Stores,No
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,...,40.80,-91.12,$18076,$36853,$112139,834,5,5311,Department Stores,No
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,...,33.18,-117.29,$16894,$34449,$36540,686,3,4829,Money Transfer,No
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,...,41.42,-87.35,$26168,$53350,$128676,685,5,4829,Money Transfer,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,...,38.86,-76.60,$33529,$68362,$96182,711,2,5813,Drinking Places (Alcoholic Beverages),No


**2. Identification des variables peu pertinentes**  
  - Liste des variables exclues  
  - Justifications (faible variance, redondance, trop de valeurs manquantes, etc.)(liste dans le pdf)  
  - Visualisations ou statistiques descriptives à l’appui  (généralement boxplot)

In [ ]:
# Check variance for numeric features
low_variance = merged_df.var(numeric_only=True).sort_values()
print(low_variance)

# Look at categorical features' unique values
for col in merged_df.select_dtypes(include=['object']).columns:
    print(col, merged_df[col].nunique())

**3. Nettoyage des données**  
  - Valeurs manquantes  
  - Doublons  
  - Types de données  
  - Incohérences  

**4. Création de variables dérivées**  
  - Heure de transaction  
  - Jour de la semaine  
  - Montant normalisé  
  - Intervalle entre deux transactions  
  - Autres pertinentes  

**5. Visualisation exploratoire**  
  - Diagrammes en secteurs des top 10 commerçants (revenus & nombre de transactions)  
  - Visualisation des périodes de pointe  
  - Interprétation  

**6. Encodage des variables catégorielles**  
  - Codes MCC, types de cartes  
  - Méthodes d’encodage testées (Label vs One-hot)  
  - Justification des choix retenus 

# 3.2  Analyse d’association (20%)


- **1. Préparation des transactions pour Apriori / FP-Growth**  
  - Structure attendue   (apriori vs fp-growth)
  - Préparation des transactions pour être compatibles avec chacune

- **2.Extraction de règles d’association**  
  - Exemples concrets  
  - Patterns observés (habitudes de consommation, fidélité, etc.)

- **3. Codes MCC et types de commerces**  
  - Dépenses élevées  
  - Patterns comportementaux  


- **4. Associations types cartes ↔ commerçants**  
  - Profils d’utilisation  

- **5.Combinaisons liées aux fraudes**  
  - Montants élevés, heures tardives, types de commerçants


- **6. interprétation des métriques**  
  - Support, confiance, lift  
  - Visualisations des règles  

- **7. Choix des seuils (support, confiance)**  
  - Justification en fonction du volume et de l’objectif  

# 3.3 Analyse de séquences (15%)

- **Construction des séquences de transactions**  
  - Par client  
  - Variables d’ordre  (horodatage, type de commer¸cant, montant, etc.)?

- **Application d’algorithmes (PrefixSpan, SPADE, EMMA)**  
  - Parcours typiques identifiés  


- **Habitudes et routines**  
  - Parcours d’achat récurrents  (fréquences, routines ou habitudes récurrentes)


- **Patterns précurseurs de fraude**  
  - Petites transactions → retrait → fraude  
  - Interprétation  par motifs temporels ou comportementaux


- **Évolution du comportement client**  
  - Séquences avec contraintes temporelles 


- **Visualisations**  
  - Graphes de séquences fréquentes  
  - Aide à l’interprétation  

# 3.4 Recommandation sur les associations ou séquences (10%)

- **Objectif : proposer des items complémentaires**  
- **Méthodes testées :**  
  - Règles d’association fréquentes  
  - Séquences typiques  
  - Similarité entre transactions  
- **Implémentation d’un script Python autonome (`recommender.py`)**  
- **Exemple de recommandations générées**  

---

# 3.5 Analyse statistique comparative des classes (10%)

- **Comparaison fraude vs légitime**  
  - Scatter plots (2D, 3D)  
  - Réduction de dimension (ACP, t-SNE, UMAP)  
  - Regroupements et anomalies  


- **Approche univariée**  
  - Sélection de variables pertinentes  
  - Tests statistiques (t-test, Mann-Whitney, KS-test)  
  - Résultats comparés  


- **Approche multivariée**  
  - Hotelling, analyse discriminante linéaire  
  - Séparation des classes  
  - Visualisations complémentaires 

- **Discussion**  
  - Avantages, limites et implications  
  - Robustesse et pertinence des méthodes  

---

# 3.6 Pr´esentation des r´esultats (20%)


- Objectif de chaque méthode (bref rappel)  
- Démarche appliquée  
- Résultats obtenus  
- Commentaires et interprétations  
- Visualisations pertinentes  
- Citations des sources utilisées (obligatoires)  